# DARPA-ARC Notebook 2: fMRI Postprocessing

## Compute Motion Summary Statistics of and Visualize Motion

In [1]:
from my_settings import (os, op, np, read_csv, version, root_dir, mri_dir, 
                         behavior_dir, task, subjects, thresholds, paradigm, 
                         fd, tr, session, modality, plt, DataFrame)
from collections import defaultdict

def demean(arr): return arr - arr.mean()

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Main loop.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

stats = defaultdict(list)

for subject in subjects:
    #
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load and prepare MRI data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    #
    ## Load gray/white matter timeseries.
    npz = np.load(op.join(root_dir, 'fmri_motion/%s_%s_qc_data.npz' % (subject, task)))
    gm = np.apply_along_axis(demean, 1, npz['gm'])
    wm = np.apply_along_axis(demean, 1, npz['wm'])
    #
    ''' A NOTE ON MOTION DATA.
    1) Understanding Freesurfer motion outputs: https://mail.nmr.mgh.harvard.edu/pipermail//freesurfer/2013-May/030273.html
    2) Understanding angular displacement: https://en.wikipedia.org/wiki/Angular_displacement
    3) Understanding framewise displacement: See Power 2012, 2014
    '''
    #
    ## Read motion data.
    mc = os.path.join(mri_dir, subject, '%s_%03d' % (task, session), '%03d' % session, 'fmcpr.mcdat')
    mc = np.loadtxt(mc)[:,1:7]
    #
    ## Invert angular displacement.
    mc[:,:3] = np.deg2rad(mc[:,:3]) # Convert degrees to radians
    mc[:,:3] *= 50                  # Convert radians to mm [Following Power 2012, we assume a head ~ sphere w/ r=50mm]
    #
    ## Compute framewise displacement (See Power 2012, 2014).
    this_fd = np.insert( np.abs( np.diff(mc, axis=0) ).sum(axis=1), 0, 0 )
    #
    ## Compute absolute displacement. 
    rot = ( np.abs( mc - mc[0] )[:,:3] ).sum(axis=1)
    trans = ( np.abs( mc - mc[0] )[:,3:] ).sum(axis=1)
    #
    ## Compute volumes to remove.
    rejections = np.zeros_like(thresholds)
    for n, threshold in enumerate(thresholds): rejections[n] += (this_fd >= threshold).sum()
    #
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Load and prepare behavior data.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    #
    ## Load behavior CSV.
    df = read_csv(op.join(behavior_dir, '%s_%s_%s_%s-%i' % (subject, task, modality, paradigm, session)))
    #
    ## Create onsets of each TR.
    onsets = np.cumsum( np.ones_like(this_fd) * tr )
    onsets = np.insert(onsets, 0, 0)
    #
    ## Define onsets/offsets of trials & shocks.
    trial_starts = df.RiskOnset.values
    trial_ends = np.append(df.FixOnset[1:], trial_starts[-1] + 5.25)
    shock_starts = df.ShockOnset[~df.ShockOnset.isnull()].values
    shock_ends = np.array([trial_ends[np.argmin((trial_ends - s)**2)] for s in shock_starts])
    #
    ## Digitize onsets/offsets.
    trial_starts = np.digitize(trial_starts, onsets)
    trial_ends = np.digitize(trial_ends, onsets)
    shock_starts = np.digitize(shock_starts, onsets)
    shock_ends = np.digitize(shock_ends, onsets)
    #
    ## Make boxcars for plotting.
    trials = np.zeros_like(this_fd)
    for i,j in zip(trial_starts, trial_ends): trials[i:j+1] += 1 
    #   
    shocks = np.zeros_like(this_fd)
    for i,j in zip(shock_starts, shock_ends): shocks[i:j+1] += 1 
    #
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    ### Calculate summary statistics.
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
    #
    stats['Subject'] += [subject]
    #
    ## Add motion information.
    stats['Abs_Disp_Rot']   += [rot.max()]
    stats['Abs_Disp_Trans'] += [trans.max()]
    stats['FD_mean'] += [this_fd.mean()]
    stats['FD_sd']   += [this_fd.std()]
    stats['FD_max']  += [this_fd.max()]
    #
    ## Calculate number of rejections.
    fd_index, = np.where(this_fd >= fd)
    n_reject = len(fd_index)
    stats['FD_reject'] += [n_reject]
    #
    ## Calculate fraction of rejected displacements across all 
    ## instances of a portion of the run.
    stats['FD_frac_task']  += [((trials) * (this_fd >= fd)).mean()]
    stats['FD_frac_rest']  += [((1 - trials) * (this_fd >= fd)).mean()]
    stats['FD_frac_shock'] += [((shocks) * (this_fd >= fd)).mean()]
    #
    ## Calculate percentages of rejection displacesments
    ## within given categories (non-shock task, shock, rest)
    stats['FD_perc_rest']    += [(1-trials)[fd_index].sum() / float(n_reject) ]
    stats['FD_perc_shock']   += [shocks[fd_index].sum() / float(n_reject) ]
    stats['FD_perc_task_ns'] += [1 - stats['FD_perc_rest'][-1] - stats['FD_perc_shock'][-1]]
    #
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
### Save summary statistics.
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
#    
stats = DataFrame(stats)
stats = stats[['Subject', 'Abs_Disp_Rot', 'Abs_Disp_Trans', 'FD_mean', 'FD_sd', 'FD_max',
               'FD_reject', 'FD_frac_task', 'FD_frac_shock', 'FD_frac_rest',
               'FD_perc_task_ns',  'FD_perc_shock', 'FD_perc_rest']]
stats.to_csv(op.join(root_dir, 'fmri_motion/motion_stats.csv'), index=False)

print('Done.')

/autofs/space/karima_001/users/alex/software/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:106: RuntimeWarning: invalid value encountered in double_scalars
/autofs/space/karima_001/users/alex/software/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:107: RuntimeWarning: invalid value encountered in double_scalars


Done.
